In [11]:
from PIL import Image
import numpy
import torch 
from torchvision.utils import save_image
from math import sqrt
from torchvision.transforms.functional import *

def minmaxnormization(tensor):
    tensor=tensor-tensor.min()
    tensor=tensor/tensor.max()
    return tensor

img=Image.open(r'.\LC08B5.TIF')
img_gray=to_tensor(img).view(7751,7591)
img_torch=img_gray[3500:4700,2300:3500]
band_fn='LC08B5.TIF'
in_ds = gdal.Open(band_fn)
for i in range(64):
    noised=torch.randn(1200,1200)
    img_torch_noised=img_torch.float()+noised*3551
#     save_image(img_torch_noised,'./imgfolder/band5_700/imgnoised_{}.png'.format(i+1))
    img_numpy_noised=img_torch_noised.numpy()
    gtiff_driver = gdal.GetDriverByName('GTiff')
    out_ds = gtiff_driver.Create(
        './imgfolder/band5_3551/imgnoised_{}.tif'.format(i+1),1200, 1200,1, 7)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform((386385.0+30*2300, 30.0, 0.0, 1873815.0-30.0*3500, 0.0, -30.0),)
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(img_numpy_noised)
    out_ds.FlushCache()
    out_band.ComputeStatistics(True)
    del out_ds
    

3551.023401280882

In [67]:
import gdal
from PIL import Image
import numpy
import torch 
from torchvision.utils import save_image
from math import sqrt
from torchvision.transforms.functional import *

def minmaxnormization(tensor):
    tensor=tensor-tensor.min()
    tensor=tensor/tensor.max()
    return tensor

band_fn='LC08B5.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
# band6=torch.from_numpy(in_band.ReadAsArray().astype('float'))[2000:6000,2000:6000]
# band6=min_max_normalization(band6,0.0,1.0)


for i in range(64):
    gtiff_driver = gdal.GetDriverByName('GTiff')
    out_ds = gtiff_driver.Create(
        './output_denoised/AE/out_denoised{}.tif'.format(i+1),1200, 1200,1, 7)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform((386385.0+30*2300, 30.0, 0.0, 1873815.0-30.0*3500, 0.0, -30.0),)
    out_band = out_ds.GetRasterBand(1)
    readyData=torch.randn(1200,1200).numpy()
    out_band.WriteArray(readyData)
    out_ds.FlushCache()
    out_band.ComputeStatistics(True)
    del out_ds
    
print('done')

done


# Record outputs for noised_1750

In [2]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms as tfs
from torchvision.utils import save_image
from PIL import Image
from torch import nn
from torch.autograd import Variable
from torchvision.transforms.functional import *
import os
import numpy
import torch 
import gdal

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(True),
            nn.Linear(32, 16),
            nn.ReLU(True),
            nn.Linear(16, 8),
            nn.ReLU(True),
            nn.Linear(8, 4),
            nn.ReLU(True),
            nn.Linear(4, 1)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(1, 4),
            nn.ReLU(True),
            nn.Linear(4, 8),
            nn.ReLU(True),
            nn.Linear(8, 16),
            nn.ReLU(True),
            nn.Linear(16, 32),
            nn.ReLU(True),
            nn.Linear(32, 64),
            nn.ReLU(True),
            nn.Tanh()
        )

    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return encode, decode
    
def minmaxnormization(tensor):
    tensor=tensor-tensor.min()
    tensor=tensor/tensor.max()
    return tensor

class LandsatDataset():

    def __init__(self, root_dir, transform=None):

        self.root_dir = root_dir
        self.transform = transform
#         self.batch_size=batch_size
    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
#         samples=[]
#         for i in range(self.batch_size):
        img_name = os.listdir(self.root_dir)[idx]#[idx*self.batch_size+i]
        in_ds = gdal.Open(self.root_dir+img_name)
        image=torch.from_numpy(in_ds.GetRasterBand(1).ReadAsArray().astype('float')).view(1200,1200)
        image=minmaxnormization(image)
        image_tensor=image.cuda()
        return image_tensor
    
    
dataset = LandsatDataset('./imgfolder/band5_1750/',transform=None)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)


model = autoencoder().cuda()
model.load_state_dict(torch.load('./model_dict/1113_autoencoder_1750_test10.pth'))
model.eval()


driverDic={}
band_fn='LC08B5.TIF'
in_ds = gdal.Open(band_fn)
for i in range(64):
    key = 'driver{}'.format(i)
    band_fn='./output_denoised/AE_1750_test10/out_denoised{}.tif'.format(i+1)
    driverDic[key]=gdal.GetDriverByName('GTiff').Create(band_fn,1200,1200, 1, 6)
    driverDic[key].SetProjection(in_ds.GetProjection())
    driverDic[key].SetGeoTransform((386385.0+30*2300, 30.0, 0.0, 1873815.0-30.0*3500, 0.0, -30.0),)
    
for data in dataloader:
    for batch in range(1200): #1200
        img_tuple=[]
        for i in range(64):
            img=data[i,batch,:].view(1,-1)
            img_tuple.append(img)
        img_tuple=tuple(img_tuple)
        img_input=torch.cat(img_tuple,0).permute(1,0).float()
        img_input=Variable(img_input)
        _,output= model(img_input) 
        out_trans=output.permute(1,0)
        for k in range(64):
            outdata=out_trans[k,:].view(1,1200)
            outdata=outdata*10000
            outdata=outdata.cpu().detach().numpy()
            out_ds = driverDic['driver{}'.format(k)]
            out_band=out_ds.GetRasterBand(1)
            out_band.WriteArray(outdata,0,batch)
            out_ds.FlushCache()
            out_band.ComputeStatistics(False)
        print('line {} done'.format(batch))
    
for j in range(64):
    out_ds = driverDic['driver{}'.format(j)]
    del out_ds
print('out_ds deleted.')
    
    
    
driverDic={}
band_fn='LC08B5.TIF'
in_ds = gdal.Open(band_fn)
for i in range(1):
    key = 'driver{}'.format(i)
    band_fn='./output_denoised/AE_1750_test10/out_encoded{}.tif'.format(i+1)
    driverDic[key]=gdal.GetDriverByName('GTiff').Create(band_fn,1200,1200, 1, 6)
    driverDic[key].SetProjection(in_ds.GetProjection())
    driverDic[key].SetGeoTransform((386385.0+30*2300, 30.0, 0.0, 1873815.0-30.0*3500, 0.0, -30.0),)
    
for batch in range(1200): #1200
    img_tuple=[]
    for i in range(64):
        img=data[i,batch,:].view(1,-1)
        img_tuple.append(img)
    img_tuple=tuple(img_tuple)
    img_input=torch.cat(img_tuple,0).permute(1,0).float()
    img_input=Variable(img_input)
    encoded,_= model(img_input) 
    encoded_trans=encoded.permute(1,0)
    for k in range(1):
        outdata=encoded_trans[k,:].view(1,1200)
        outdata=outdata*10000
        outdata=outdata.cpu().detach().numpy()
        out_ds = driverDic['driver{}'.format(k)]
        out_band=out_ds.GetRasterBand(1)
        out_band.WriteArray(outdata,0,batch)
        out_ds.FlushCache()
        out_band.ComputeStatistics(False)
    print('line {} done'.format(batch))
                
for j in range(1):
    out_ds = driverDic['driver{}'.format(j)]
    del out_ds
print('out_ds deleted.')

line 0 done
line 1 done
line 2 done
line 3 done
line 4 done
line 5 done
line 6 done
line 7 done
line 8 done
line 9 done
line 10 done
line 11 done
line 12 done
line 13 done
line 14 done
line 15 done
line 16 done
line 17 done
line 18 done
line 19 done
line 20 done
line 21 done
line 22 done
line 23 done
line 24 done
line 25 done
line 26 done
line 27 done
line 28 done
line 29 done
line 30 done
line 31 done
line 32 done
line 33 done
line 34 done
line 35 done
line 36 done
line 37 done
line 38 done
line 39 done
line 40 done
line 41 done
line 42 done
line 43 done
line 44 done
line 45 done
line 46 done
line 47 done
line 48 done
line 49 done
line 50 done
line 51 done
line 52 done
line 53 done
line 54 done
line 55 done
line 56 done
line 57 done
line 58 done
line 59 done
line 60 done
line 61 done
line 62 done
line 63 done
line 64 done
line 65 done
line 66 done
line 67 done
line 68 done
line 69 done
line 70 done
line 71 done
line 72 done
line 73 done
line 74 done
line 75 done
line 76 done
line 77 d

line 593 done
line 594 done
line 595 done
line 596 done
line 597 done
line 598 done
line 599 done
line 600 done
line 601 done
line 602 done
line 603 done
line 604 done
line 605 done
line 606 done
line 607 done
line 608 done
line 609 done
line 610 done
line 611 done
line 612 done
line 613 done
line 614 done
line 615 done
line 616 done
line 617 done
line 618 done
line 619 done
line 620 done
line 621 done
line 622 done
line 623 done
line 624 done
line 625 done
line 626 done
line 627 done
line 628 done
line 629 done
line 630 done
line 631 done
line 632 done
line 633 done
line 634 done
line 635 done
line 636 done
line 637 done
line 638 done
line 639 done
line 640 done
line 641 done
line 642 done
line 643 done
line 644 done
line 645 done
line 646 done
line 647 done
line 648 done
line 649 done
line 650 done
line 651 done
line 652 done
line 653 done
line 654 done
line 655 done
line 656 done
line 657 done
line 658 done
line 659 done
line 660 done
line 661 done
line 662 done
line 663 done
line 6

line 1166 done
line 1167 done
line 1168 done
line 1169 done
line 1170 done
line 1171 done
line 1172 done
line 1173 done
line 1174 done
line 1175 done
line 1176 done
line 1177 done
line 1178 done
line 1179 done
line 1180 done
line 1181 done
line 1182 done
line 1183 done
line 1184 done
line 1185 done
line 1186 done
line 1187 done
line 1188 done
line 1189 done
line 1190 done
line 1191 done
line 1192 done
line 1193 done
line 1194 done
line 1195 done
line 1196 done
line 1197 done
line 1198 done
line 1199 done
out_ds deleted.
line 0 done
line 1 done
line 2 done
line 3 done
line 4 done
line 5 done
line 6 done
line 7 done
line 8 done
line 9 done
line 10 done
line 11 done
line 12 done
line 13 done
line 14 done
line 15 done
line 16 done
line 17 done
line 18 done
line 19 done
line 20 done
line 21 done
line 22 done
line 23 done
line 24 done
line 25 done
line 26 done
line 27 done
line 28 done
line 29 done
line 30 done
line 31 done
line 32 done
line 33 done
line 34 done
line 35 done
line 36 done
lin

line 555 done
line 556 done
line 557 done
line 558 done
line 559 done
line 560 done
line 561 done
line 562 done
line 563 done
line 564 done
line 565 done
line 566 done
line 567 done
line 568 done
line 569 done
line 570 done
line 571 done
line 572 done
line 573 done
line 574 done
line 575 done
line 576 done
line 577 done
line 578 done
line 579 done
line 580 done
line 581 done
line 582 done
line 583 done
line 584 done
line 585 done
line 586 done
line 587 done
line 588 done
line 589 done
line 590 done
line 591 done
line 592 done
line 593 done
line 594 done
line 595 done
line 596 done
line 597 done
line 598 done
line 599 done
line 600 done
line 601 done
line 602 done
line 603 done
line 604 done
line 605 done
line 606 done
line 607 done
line 608 done
line 609 done
line 610 done
line 611 done
line 612 done
line 613 done
line 614 done
line 615 done
line 616 done
line 617 done
line 618 done
line 619 done
line 620 done
line 621 done
line 622 done
line 623 done
line 624 done
line 625 done
line 6

line 1130 done
line 1131 done
line 1132 done
line 1133 done
line 1134 done
line 1135 done
line 1136 done
line 1137 done
line 1138 done
line 1139 done
line 1140 done
line 1141 done
line 1142 done
line 1143 done
line 1144 done
line 1145 done
line 1146 done
line 1147 done
line 1148 done
line 1149 done
line 1150 done
line 1151 done
line 1152 done
line 1153 done
line 1154 done
line 1155 done
line 1156 done
line 1157 done
line 1158 done
line 1159 done
line 1160 done
line 1161 done
line 1162 done
line 1163 done
line 1164 done
line 1165 done
line 1166 done
line 1167 done
line 1168 done
line 1169 done
line 1170 done
line 1171 done
line 1172 done
line 1173 done
line 1174 done
line 1175 done
line 1176 done
line 1177 done
line 1178 done
line 1179 done
line 1180 done
line 1181 done
line 1182 done
line 1183 done
line 1184 done
line 1185 done
line 1186 done
line 1187 done
line 1188 done
line 1189 done
line 1190 done
line 1191 done
line 1192 done
line 1193 done
line 1194 done
line 1195 done
line 1196 